In [1]:
import os
from dotenv import load_dotenv
import mlflow

# Load environment variables from .env file
load_dotenv()
print(os.getenv('AWS-MLFLOW'))
# Get the AWS MLflow URI from .env file
aws_mlflow_uri = os.getenv('AWS-MLFLOW')

mlflow.set_tracking_uri(aws_mlflow_uri)
with mlflow.start_run():
    mlflow.log_param("param1", 15)
    mlflow.log_metric("metric1", 0.89)

http://ec2-54-86-71-188.compute-1.amazonaws.com:5000/
🏃 View run thundering-mole-678 at: http://ec2-54-86-71-188.compute-1.amazonaws.com:5000/#/experiments/0/runs/2ee75897e41b4b9ea5e11846d4833b0a
🧪 View experiment at: http://ec2-54-86-71-188.compute-1.amazonaws.com:5000/#/experiments/0


In [2]:
# Set or create an experiment
mlflow.set_experiment("Exp 6 - LightGBM HP Tuning")

2025/07/05 19:21:41 INFO mlflow.tracking.fluent: Experiment with name 'Exp 6 - LightGBM HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://ytsentimentarif/531807705718017736', creation_time=1751732500866, experiment_id='531807705718017736', last_update_time=1751732500866, lifecycle_stage='active', name='Exp 6 - LightGBM HP Tuning', tags={}>

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt

c:\Program Files\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
data = pd.read_csv('reddit_preprocessed.csv').dropna()
data.shape

(36878, 2)

In [6]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
data['category'] = data['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
data = data.dropna(subset=['category'])

In [ ]:
ngram_range = (1, 3)
max_features = 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(data['clean_comment'])
y = data['category']

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

In [11]:
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, params, trial_number):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"Trial_{trial_number}_{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        mlflow.log_param("algo_name", model_name)

        for key, value in params.items():
            mlflow.log_param(key, value)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        mlflow.sklearn.log_model(model, f"{model_name}_model")

        return accuracy

In [9]:
def objective_lightgbm(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 15)
    num_leaves = trial.suggest_int('num_leaves', 20, 150)
    min_child_samples = trial.suggest_int('min_child_samples', 10, 100)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    reg_alpha = trial.suggest_float('reg_alpha', 1e-4, 10.0, log=True)  # L1 regularization
    reg_lambda = trial.suggest_float('reg_lambda', 1e-4, 10.0, log=True)  # L2 regularization

    params = {
        'n_estimators': n_estimators,
        'learning_rate': learning_rate,
        'max_depth': max_depth,
        'num_leaves': num_leaves,
        'min_child_samples': min_child_samples,
        'colsample_bytree': colsample_bytree,
        'subsample': subsample,
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda
    }

    model = LGBMClassifier(n_estimators=n_estimators,
                           learning_rate=learning_rate,
                           max_depth=max_depth,
                           num_leaves=num_leaves,
                           min_child_samples=min_child_samples,
                           colsample_bytree=colsample_bytree,
                           subsample=subsample,
                           reg_alpha=reg_alpha,
                           reg_lambda=reg_lambda,
                           random_state=42)

    accuracy = log_mlflow("LightGBM", model, X_train, X_test, y_train, y_test, params, trial.number)

    return accuracy

In [10]:
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lightgbm, n_trials=10) 

    best_params = study.best_params
    best_model = LGBMClassifier(n_estimators=best_params['n_estimators'],
                                learning_rate=best_params['learning_rate'],
                                max_depth=best_params['max_depth'],
                                num_leaves=best_params['num_leaves'],
                                min_child_samples=best_params['min_child_samples'],
                                colsample_bytree=best_params['colsample_bytree'],
                                subsample=best_params['subsample'],
                                reg_alpha=best_params['reg_alpha'],
                                reg_lambda=best_params['reg_lambda'],
                                random_state=42)

    log_mlflow("LightGBM", best_model, X_train, X_test, y_train, y_test, best_params, "Best")

    optuna.visualization.plot_param_importances(study).show()

    optuna.visualization.plot_optimization_history(study).show()

In [12]:
run_optuna_experiment()

[I 2025-07-05 19:30:47,235] A new study created in memory with name: no-name-0264e86f-8c3a-436f-a1fe-c3d0f602e6de


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023497 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 99167
[LightGBM] [Info] Number of data points in the train set: 37989, number of used features: 966
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

c:\Program Files\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2025/07/05 19:31:24 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/05 19:31:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_0_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-54-86-71-188.compute-1.amazonaws.com:5000/#/experiments/531807705718017736/runs/b9f9de383a104258aa5862e8aba4c9bf
🧪 View experiment at: http://ec2-54-86-71-188.compute-1.amazonaws.com:5000/#/experiments/531807705718017736


[I 2025-07-05 19:31:55,345] Trial 0 finished with value: 0.8105917035165298 and parameters: {'n_estimators': 852, 'learning_rate': 0.08621571446273696, 'max_depth': 6, 'num_leaves': 29, 'min_child_samples': 45, 'colsample_bytree': 0.818089039557262, 'subsample': 0.5729356407603989, 'reg_alpha': 0.4593021026687401, 'reg_lambda': 0.6840363548979358}. Best is trial 0 with value: 0.8105917035165298.


...

[I 2025-07-05 19:42:32,334] Trial 9 finished with value: 0.6131817224678879 and parameters: {'n_estimators': 471, 'learning_rate': 0.0007109696155602584, 'max_depth': 7, 'num_leaves': 44, 'min_child_samples': 12, 'colsample_bytree': 0.9476806178537832, 'subsample': 0.962504671152989, 'reg_alpha': 0.009431187518203932, 'reg_lambda': 4.378728759362802}. Best is trial 5 with value: 0.8132238365971783.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.029671 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 99252
[LightGBM] [Info] Number of data points in the train set: 37989, number of used features: 976
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

c:\Program Files\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2025/07/05 19:43:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/05 19:43:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_Best_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-54-86-71-188.compute-1.amazonaws.com:5000/#/experiments/531807705718017736/runs/7c07cb97ae6a48179eb76222a0ece8a1
🧪 View experiment at: http://ec2-54-86-71-188.compute-1.amazonaws.com:5000/#/experiments/531807705718017736


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed